This notebook is for sanity check about the FED3 device data.

The ``Motor_Turns`` column implies the status of the FED3. Though there is no official cutoff for its value when the device get stuck, we use an empirical values based on our data to check the ratio of pellets happens during dispense (or not well-feedbacked periods).

We choose 15 as a not good pellet release and count the percentage of such bad releases among all pellet release events in a session.

When the percentage of bad pellet releases are too many (> 30% or > 25% based on different data), we drop that sample.

In [1]:
import sys
sys.path.insert(0, '../scripts')

import pandas as pd
import json
from path import fr1_ctrl_path, fr1_cask_path, fr1_ctrl_sheets, fr1_cask_sheets, \
                rev_ctrl_path, rev_cask_path, rev_ctrl_sheets, rev_cask_sheets
dispense_stats = {}

In [2]:
def plot_motor_turn(excel_path, sheet):
    data = pd.read_excel(excel_path, sheet_name=sheet)
    data = data[(data['Event'] == 'Pellet') & (data['Motor_Turns'] > 1)][['Motor_Turns', 'Pellet_Count']]
    exceeding_cnt = int((data['Motor_Turns'] > 15).astype(int).sum())
    return exceeding_cnt, round(exceeding_cnt / len(data), 3)

### FR1 ctrl

In [3]:
temp_dict = {}
for sheet in fr1_ctrl_sheets[:]:
    cnt, prop = plot_motor_turn(fr1_ctrl_path, sheet)
    temp_dict[sheet] = [cnt, prop]
dispense_stats['FR1_ctrl'] = temp_dict

temp_dict = {}
for sheet in fr1_cask_sheets:
    cnt, prop = plot_motor_turn(fr1_cask_path, sheet)
    temp_dict[sheet] = [cnt, prop]
dispense_stats['FR1_cask'] = temp_dict

temp_dict = {}
for sheet in rev_ctrl_sheets:
    cnt, prop = plot_motor_turn(rev_ctrl_path, sheet)
    temp_dict[sheet] = [cnt, prop]
dispense_stats['rev_ctrl'] = temp_dict

temp_dict = {}
for sheet in rev_cask_sheets:
    cnt, prop = plot_motor_turn(rev_cask_path, sheet)
    temp_dict[sheet] = [cnt, prop]
dispense_stats['rev_cask'] = temp_dict

In [4]:
with open('../stats/cask_dispense.json', 'w') as f:
    json.dump(dispense_stats, f)